<a href="https://colab.research.google.com/github/aydinmyilmaz/Transformers-Tutorials/blob/master/sentiment_fine_tuning_huggingface_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 41.6 MB/s 
     |████████████████████████████████| 77 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 


In [2]:
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /content/sarcasm.json

--2022-04-11 07:41:56--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/content/sarcasm.json’

/content/sarcasm.js 100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2022-04-11 07:41:56 (179 MB/s) - ‘/content/sarcasm.json’ saved [5643545/5643545]



In [4]:
training_size = 20000

with open("/content/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [5]:
training_sentences[0:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [6]:
print(len(training_sentences))
print(len(validation_sentences))

20000
6709


In [7]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True,
                            return_tensors='tf')
val_encodings = tokenizer(validation_sentences,
                          truncation=True,
                          padding=True,
                          return_tensors='tf')

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [11]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
test_sentence = "former versace store clerk sues over secret 'black code' for minority shoppers"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

[0.53186345 0.46813658]
[0.5347489  0.46525115]


In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [14]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
1250/1250 [==============================] - 352s 273ms/step - loss: 0.2632 - accuracy: 0.8887 - val_loss: 0.2036 - val_accuracy: 0.9191
Epoch 2/3
1250/1250 [==============================] - 339s 271ms/step - loss: 0.0925 - accuracy: 0.9667 - val_loss: 0.2557 - val_accuracy: 0.9191
Epoch 3/3
1250/1250 [==============================] - 341s 272ms/step - loss: 0.0404 - accuracy: 0.9869 - val_loss: 0.3280 - val_accuracy: 0.9146


In [15]:
model.save_pretrained("/content/sentiment_custom_model")

In [ ]:
#### Load saved model and run predict function

In [16]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("/content/sentiment_custom_model")

Some layers from the model checkpoint at /content/sentiment_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentiment_custom_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [18]:
test_sentence ="former versace store clerk sues over secret 'black code' for minority shoppers"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

[0.999385 0.000615]
[0.003198 0.996802]
